# 자율주행 키워드 크롤링 

- 칼럼정리 
1. 기업이름
2. 고용형태
3. 연봉
4. 기업주소
5. 키워드 
6. 담당업무 (분야)
7. 필요역량 (자격조건 + 우대사항)

### import 목록

In [12]:
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import urllib



### 웹사이트 연결

In [3]:
url = "https://job.incruit.com/jobdb_list/searchjob.asp?kw=%C0%DA%C0%B2%C1%D6%C7%E0&rgn2=11&rgn2=18&page={page_number}"        # 자신의 찾으려는 웹사이트 입력
req = Request(url.format(page_number = urllib.parse.quote('1')))
res = urlopen(req)
soup = BeautifulSoup(res, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html lang="ko">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="text/javascript" http-equiv="Content-Script-Type"/>
  <meta content="text/css" http-equiv="Content-Style-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>
   서울특별시,경기도 자율주행 :: 인크루트 채용정보
  </title>
  <meta content=" 서울특별시,경기도 자율주행 인크루트 채용정보(incruit.com) - 믿을 수 있는 취업정보사이트, 경력별, 지역별, 직종별 구인구직정보, 직업별 일자리정보, 실시간 채용정보, 기업별 입사비법" name="Description"/>
  <meta content=" 서울특별시,경기도 자율주행,취업정보사이트,구인구직사이트,직업,일자리사이트,구인구직사이트,채용정보사이트,인턴,공기업채용,구인광고,구직정보,채용공고,대기업채용정보,여성일자리,기업정보,공사채용,취직,JOB,잡,인크루트,incruit" name="Keywords"/>
  <meta content="https://job.incruit.com/jobdb_list/searchjob.asp?kw=%C0%DA%C0%B2%C1%D6%C7%E0&amp;rgn2=11&amp;rgn2=18" property="og:url"/>
  <meta content=" 서울특별시,경기도 자율주행 :: 인크루트 채용정보" property="og:title"/>
  <meta content=" 서울특별시,경기도 자율주행 인크루트 채용정보(incruit.com) - 믿을 수 있는 취업정보사이트, 경력별, 지역별, 직종별 구인구직정보, 직업별 일자리정보, 실시간 채용정보, 기업별 입사비법" 

### 한페이지 크롤링 테스트

In [56]:
element = soup.find_all('ul',class_='c_row')  #  하위 클래스 설정

for idx, each in enumerate(element):
    cl_top = each.find('div',class_='cl_top')
    cell_mid = each.find('div',class_='cell_mid')
    
    print(str(idx))
    print(each.find('a',class_='cpname').string)    # 회사이름
    print('')
    print(cell_mid.find(class_='cl_top').get_text())    # 모집명
    print('')
    print(cell_mid.find('a')['href'])                   # 링크
    print('')
    print(cell_mid.find(class_='cl_md').get_text())     # 경력 학력 주소 연봉
    print('')
    print(cell_mid.find(class_='cl_btm').get_text())    # 분야
    print('')
    print('--------------------------------------')

0
(주)에이알247


자율주행 로봇 Perception 개발자 채용
스크랩


https://job.incruit.com/jobdb_info/jobpost.asp?job=2205190006235&src=gsw*etc


경력(연차무관) 대졸이상
경기 성남시 정규직 6,000~7,000만원



빅데이터·AI(인공지능), 응용프로그래머, 소프트웨어·하드웨어


--------------------------------------
1
쓰리핸즈


[ 자율주행 Quality Assurance(QA) ] 자율주행 로봇 배달 플랫폼
스크랩


https://job.incruit.com/jobdb_info/jobpost.asp?job=2312170000111&src=gsw*etc


경력1~5년 대졸
서울 성동구 정규직



소프트웨어·하드웨어, 게임(Game), 응용프로그래머


--------------------------------------
2
매버릭컨설팅(주)


PROJECT MANAGER (자동차 자율주행, ADAS)
스크랩


https://job.incruit.com/jobdb_info/jobpost.asp?job=2212230005310&src=gsw*etc


경력8~22년 대졸
경기 안양시 동안구 외 정규직



경영·기획·전략


--------------------------------------
3
현대자동차(주)


자율주행 및 자율주차 시험
스크랩


https://job.incruit.com/jobdb_info/jobpost.asp?job=2401100000202&src=gsw*etc


경력(연차무관) 대졸이상
경기 화성시 정규직



빅데이터·AI(인공지능), 품질관리, 기계·금속·재료


--------------------------------------
4
(주)에이알247


자율주행 SW 엔지니어 신입/경력 정규직 채용
스크랩


https://job.incruit.com/jobdb_info/jobpost.asp?job=

### 1차크롤링

In [57]:
company_list = []

for i in range(1,4):
    url = "https://job.incruit.com/jobdb_list/searchjob.asp?kw=%C0%DA%C0%B2%C1%D6%C7%E0&rgn2=11&rgn2=18&page={page_number}"        # 자신의 찾으려는 웹사이트 입력
    req = Request(url.format(page_number = urllib.parse.quote(str(i))))
    res = urlopen(req)
    soup = BeautifulSoup(res, "html.parser")
    
    element = soup.find_all('ul',class_='c_row')
    for each in element:
        cl_top = each.find('div',class_='cl_top')
        cell_mid = each.find('div',class_='cell_mid')

        company = []
        name =each.find('a',class_='cpname').string    # 회사이름
        title = cell_mid.find(class_='cl_top').get_text()   # 모집명
        link = cell_mid.find('a')['href']                  # 링크
        info = cell_mid.find(class_='cl_md').get_text()     # 경력 학력 주소 연봉
        field = cell_mid.find(class_='cl_btm').get_text()  # 분야

        company.append(name)
        company.append(title)
        company.append(link)
        company.append(info)
        company.append(field)
        company_list.append(company)





In [58]:
company_list

[['(주)에이알247',
  '\n자율주행 로봇 Perception 개발자 채용\n스크랩\n',
  'https://job.incruit.com/jobdb_info/jobpost.asp?job=2205190006235&src=gsw*etc',
  '\n경력(연차무관) 대졸이상\n경기 성남시 정규직 6,000~7,000만원\n',
  '\n빅데이터·AI(인공지능), 응용프로그래머, 소프트웨어·하드웨어\n'],
 ['쓰리핸즈',
  '\n[ 자율주행 Quality Assurance(QA) ] 자율주행 로봇 배달 플랫폼\n스크랩\n',
  'https://job.incruit.com/jobdb_info/jobpost.asp?job=2312170000111&src=gsw*etc',
  '\n경력1~5년 대졸\n서울 성동구 정규직\n',
  '\n소프트웨어·하드웨어, 게임(Game), 응용프로그래머\n'],
 ['매버릭컨설팅(주)',
  '\nPROJECT MANAGER (자동차 자율주행, ADAS)\n스크랩\n',
  'https://job.incruit.com/jobdb_info/jobpost.asp?job=2212230005310&src=gsw*etc',
  '\n경력8~22년 대졸\n경기 안양시 동안구 외 정규직\n',
  '\n경영·기획·전략\n'],
 ['현대자동차(주)',
  '\n자율주행 및 자율주차 시험\n스크랩\n',
  'https://job.incruit.com/jobdb_info/jobpost.asp?job=2401100000202&src=gsw*etc',
  '\n경력(연차무관) 대졸이상\n경기 화성시 정규직\n',
  '\n빅데이터·AI(인공지능), 품질관리, 기계·금속·재료\n'],
 ['현대자동차(주)',
  '\n자율주행 및 자율주차 설계\n스크랩\n',
  'https://job.incruit.com/jobdb_info/jobpost.asp?job=2401100000190&src=gsw*etc',
  '\n경력(연차무관) 대졸이상\n

### 데이터프레임 구축

In [59]:
import pandas as pd 
autocar_df = pd.DataFrame(company_list, columns=['회사명', '모집명','링크','info','분야'])


### 데이터 가공

In [60]:
autocar_df['a'] = autocar_df['info'].str.split('\n').str[1]
autocar_df['b'] = autocar_df['info'].str.split('\n').str[2]
autocar_df['모집명'] = autocar_df['모집명'].str.split('\n').str[1]
autocar_df['분야'] = autocar_df['분야'].str.replace('\n', '')

In [61]:
autocar_df['경력'] = autocar_df['a'].str.split(' ').str[0]
autocar_df['학력']= autocar_df['a'].str.split(' ').str[1]


In [62]:
autocar_df.drop(columns={'info','a'},inplace=True)

In [63]:
import re

autocar_df['연봉'] = autocar_df['b'].apply(lambda x: ''.join(re.findall(r'[\d,~]+', x)) if re.search(r'[\d,~]', x) else None)

In [64]:
autocar_df.drop(columns={'b'},inplace=True)

In [65]:
autocar_df

,회사명,모집명,링크,분야,경력,학력,연봉
0,(주)에이알247,자율주행 로봇 Perception 개발자 채용,https://job.incruit.com/jobdb_info/jobpost.asp...,"빅데이터·AI(인공지능), 응용프로그래머, 소프트웨어·하드웨어",경력(연차무관),대졸이상,"6,000~7,000"
1,쓰리핸즈,[ 자율주행 Quality Assurance(QA) ] 자율주행 로봇 배달 플랫폼,https://job.incruit.com/jobdb_info/jobpost.asp...,"소프트웨어·하드웨어, 게임(Game), 응용프로그래머",경력1~5년,대졸,None
2,매버릭컨설팅(주),"PROJECT MANAGER (자동차 자율주행, ADAS)",https://job.incruit.com/jobdb_info/jobpost.asp...,경영·기획·전략,경력8~22년,대졸,None
3,현대자동차(주),자율주행 및 자율주차 시험,https://job.incruit.com/jobdb_info/jobpost.asp...,"빅데이터·AI(인공지능), 품질관리, 기계·금속·재료",경력(연차무관),대졸이상,None
4,현대자동차(주),자율주행 및 자율주차 설계,https://job.incruit.com/jobdb_info/jobpost.asp...,"빅데이터·AI(인공지능), ERP·시스템분석·설계",경력(연차무관),대졸이상,None
...,...,...,...,...,...,...,...
117,한국폴리텍대학 성남캠퍼스,[전액무료] 하이테크 과정 교육생 2차 모집,https://job.incruit.com/jobdb_info/jobpost.asp...,"연구원·연구개발, 기술영업, 바이오·식품공학, 빅데이터·AI(인공지능), 응용프로그...",신입/경력(연차무관),학력무관,None
118,(주) 실리콘스튜디오코리아,"2024년 상반기 실리콘스튜디오코리아 부문별 채용[가상현실 엔지니어,AI 엔지니어]",https://job.incruit.com/jobdb_info/jobpost.asp...,"응용프로그래머, 네트워크·서버·보안, 빅데이터·AI(인공지능), ERP·시스템분석·...",신입/경력(연차무관),고졸이상,None
119,한국교통안전공단,자동차안전연구원 자율주행자동차사고조사위원회 사무국 청년인턴 채용 공고,https://job.incruit.com/jobdb_info/jobpost.asp...,"경영·기획·전략, 일반사무·사무지원, 세무·회계, 법률·법무·회계",신입,학력무관,None
120,현대엠시스템즈(주),"자율주행, 센서퓨전, 임베디드, AI분야 개발자 채용",https://job.incruit.com/jobdb_info/jobpost.asp...,"전자·반도체, 시스템프로그래머",경력2년↑,초대졸,"3,000~8,000"


In [66]:
autocar_df.to_csv("../../data/eda_project_data/autocar_temp.csv", sep=',', encoding="euc-kr")

### 2차크롤링 테스트 진행 (링크, 주소 정보 가져오기)

In [55]:
url = 'https://job.incruit.com/jobdb_info/jobpost.asp?job=2312170000111&src=gsw*etc'
page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

company = soup.find(class_='jcinfo_list last')
a1 =company.find_all('div', class_='txt')
a2 = a1[-2]
address = a2.find('em').text

r1 = soup.find_all('div',class_='section_view_layout')[1]
r2 = r1.find('iframe',id='ifrmJobCont')['src']

url1='https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2312170000111'
page1 = urlopen(url1)
soup1 = BeautifulSoup(page1, 'html.parser')

r = soup1.find('tbody').get_text()

print(address)
print('---')
print(r2)
print('---')
print(r)


(06746) 서울특별시 서초구 바우뫼로 222 (양재동,영진빌딩)
---
https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2312170000111
---


자율주행 Quality Assurance(QA)

[담당업무]○ 무인 배달 로봇의 자율주행 알고리즘 및 시스템의 현장 검증 및 리포팅○ 다양한 환경에서의 반복 테스트를 통한 엣지 케이스 검출 및 리포팅○ 현장 테스트를 통한 자율주행 SW 시스템 기능의 UI/UX 피드백○ 신규 서비스 구역에 대한 자율주행 매핑 및 주행 검증


[자격요건]○ 만 1년 이상 5년 미만의 QA 업무 경력을 보유하신 분○ 테스트 케이스 작성 및 개선 경험○ 리눅스 시스템에 친숙하신 분○ 팀원과의 원활한 소통 및 협업 능력 보유[우대사항]○ 로보틱스, 컴퓨터공학 전공 혹은 그에 준하는 소프트웨어공학 기반 지식 보유○ Linux, ROS, 실내/외 자율주행 알고리즘에 대한 기본 지식 보유○ 테스트 자동화 구축 경험 또는 Python 등 스크립트 언어 작성 경험 있으신 분○ 소통을 위한 협업 tool에 능통하신 분 (Jira, Confluence)

0 명




### 2차 크롤링

In [13]:
from tqdm import tqdm 
from user_agent import generate_user_agent

address_list = []
link2_list = []

for idx, row in tqdm(autocar_df.iterrows(), total=len(autocar_df.index)):
    req = Request(url=row['링크'], headers={'User-Agent':generate_user_agent()})
    page = urlopen(req)
    soup = BeautifulSoup(page, 'html.parser')

    company = soup.find(class_='jcinfo_list last')
    a1 =company.find_all('div', class_='txt')
    a2 = a1[-2]
    address = a2.find('em').text

    link = soup.find_all('div',class_='section_view_layout')[1]
    link2 = link.find('iframe',id='ifrmJobCont')['src']

    address_list.append(address)
    link2_list.append(link2)
    

100%|██████████| 122/122 [01:38<00:00,  1.23it/s]


In [14]:
address_list, link2_list

([],
 ['https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2205190006235',
  'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2312170000111',
  'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2212230005310',
  'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2401100000202',
  'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2401100000190',
  'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2108170000567',
  'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2208300004159',
  'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2312170000114',
  'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2401100000210',
  'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2302020003951',
  'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2312170000086',
  'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2312170000100',
  'https://job.incruit.com/s_common

### 데이터 삽입

In [15]:
autocar_df['주소'] = address_list
autocar_df['링크2'] = link2_list

In [92]:
autocar_df.to_csv("../../data/eda_project_data/autocar_temp1.csv", sep=',', encoding="euc-kr")

UnicodeEncodeError: 'euc_kr' codec can't encode character '\u2022' in position 163: illegal multibyte sequence

### 키워트 삽입

In [83]:
cate_list = []
for i in range(1,123):
    cate_list.append('자율주행')

In [84]:
len(cate_list)

122

In [85]:
autocar_df['키워드'] = cate_list

In [96]:
autocar_df.to_csv("../../data/eda_project_data/autocar2.csv", encoding='utf-8')

In [5]:
import pandas as pd
autocar_df = pd.read_csv("../../data/eda_project_data/autocar2.csv")

In [16]:
autocar_df

,Unnamed: 0,회사명,모집명,링크,분야,경력,학력,연봉,주소,링크2,키워드
0,0,(주)에이알247,자율주행 로봇 Perception 개발자 채용,https://job.incruit.com/jobdb_info/jobpost.asp...,"빅데이터·AI(인공지능), 응용프로그래머, 소프트웨어·하드웨어",경력(연차무관),대졸이상,"6,000~7,000","(13636) 경기도 성남시 분당구 성남대로 69 (구미동,로드랜드EZ타워) 1층",https://job.incruit.com/s_common/jobpost/jobpo...,자율주행
1,1,쓰리핸즈,[ 자율주행 Quality Assurance(QA) ] 자율주행 로봇 배달 플랫폼,https://job.incruit.com/jobdb_info/jobpost.asp...,"소프트웨어·하드웨어, 게임(Game), 응용프로그래머",경력1~5년,대졸,NaN,"(06746) 서울특별시 서초구 바우뫼로 222 (양재동,영진빌딩)",https://job.incruit.com/s_common/jobpost/jobpo...,자율주행
2,2,매버릭컨설팅(주),"PROJECT MANAGER (자동차 자율주행, ADAS)",https://job.incruit.com/jobdb_info/jobpost.asp...,경영·기획·전략,경력8~22년,대졸,NaN,"(06628) 서울특별시 서초구 강남대로 309 2016호(서초동,코리아비지니스센타)",https://job.incruit.com/s_common/jobpost/jobpo...,자율주행
3,3,현대자동차(주),자율주행 및 자율주차 시험,https://job.incruit.com/jobdb_info/jobpost.asp...,"빅데이터·AI(인공지능), 품질관리, 기계·금속·재료",경력(연차무관),대졸이상,NaN,(06797) 서울특별시 서초구 헌릉로 12 (양재동),https://job.incruit.com/s_common/jobpost/jobpo...,자율주행
4,4,현대자동차(주),자율주행 및 자율주차 설계,https://job.incruit.com/jobdb_info/jobpost.asp...,"빅데이터·AI(인공지능), ERP·시스템분석·설계",경력(연차무관),대졸이상,NaN,(06797) 서울특별시 서초구 헌릉로 12 (양재동),https://job.incruit.com/s_common/jobpost/jobpo...,자율주행
...,...,...,...,...,...,...,...,...,...,...,...
117,117,한국폴리텍대학 성남캠퍼스,[전액무료] 하이테크 과정 교육생 2차 모집,https://job.incruit.com/jobdb_info/jobpost.asp...,"연구원·연구개발, 기술영업, 바이오·식품공학, 빅데이터·AI(인공지능), 응용프로그...",신입/경력(연차무관),학력무관,NaN,(13122) 경기 성남시 수정구 수정로 398 (산성동),https://job.incruit.com/s_common/jobpost/jobpo...,자율주행
118,118,(주) 실리콘스튜디오코리아,"2024년 상반기 실리콘스튜디오코리아 부문별 채용[가상현실 엔지니어,AI 엔지니어]",https://job.incruit.com/jobdb_info/jobpost.asp...,"응용프로그래머, 네트워크·서버·보안, 빅데이터·AI(인공지능), ERP·시스템분석·...",신입/경력(연차무관),고졸이상,NaN,(05854) 서울특별시 송파구 송파대로 201 송파 테라타워2 A동 1301호,https://job.incruit.com/s_common/jobpost/jobpo...,자율주행
119,119,한국교통안전공단,자동차안전연구원 자율주행자동차사고조사위원회 사무국 청년인턴 채용 공고,https://job.incruit.com/jobdb_info/jobpost.asp...,"경영·기획·전략, 일반사무·사무지원, 세무·회계, 법률·법무·회계",신입,학력무관,NaN,NaN,https://job.incruit.com/s_common/jobpost/jobpo...,자율주행
120,120,현대엠시스템즈(주),"자율주행, 센서퓨전, 임베디드, AI분야 개발자 채용",https://job.incruit.com/jobdb_info/jobpost.asp...,"전자·반도체, 시스템프로그래머",경력2년↑,초대졸,"3,000~8,000",NaN,https://job.incruit.com/s_common/jobpost/jobpo...,자율주행


### 3차 크롤링 테스트

In [26]:
url1='https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2312170000111'
page1 = urlopen(url1)
soup1 = BeautifulSoup(page1, 'html.parser')

r = soup1.find('tbody').get_text()
# r1 = r.find('td',class_='text-left.text-top')
# r2 = r.find(class_='text-left.text-top')[1]

print(r)
# print(r2)



자율주행 Quality Assurance(QA)

[담당업무]○ 무인 배달 로봇의 자율주행 알고리즘 및 시스템의 현장 검증 및 리포팅○ 다양한 환경에서의 반복 테스트를 통한 엣지 케이스 검출 및 리포팅○ 현장 테스트를 통한 자율주행 SW 시스템 기능의 UI/UX 피드백○ 신규 서비스 구역에 대한 자율주행 매핑 및 주행 검증


[자격요건]○ 만 1년 이상 5년 미만의 QA 업무 경력을 보유하신 분○ 테스트 케이스 작성 및 개선 경험○ 리눅스 시스템에 친숙하신 분○ 팀원과의 원활한 소통 및 협업 능력 보유[우대사항]○ 로보틱스, 컴퓨터공학 전공 혹은 그에 준하는 소프트웨어공학 기반 지식 보유○ Linux, ROS, 실내/외 자율주행 알고리즘에 대한 기본 지식 보유○ 테스트 자동화 구축 경험 또는 Python 등 스크립트 언어 작성 경험 있으신 분○ 소통을 위한 협업 tool에 능통하신 분 (Jira, Confluence)

0 명




### 3차 크롤링 진행 
1. 찾지 못하는 값 발견, 출력결과 싪패
2. 찾지 못하는 값 null값으로 입력 

In [29]:
from tqdm import tqdm 
from user_agent import generate_user_agent

task_list = []

for idx, row in tqdm(autocar_df.iterrows(), total=len(autocar_df.index)):
    req = Request(url=row['링크2'], headers={'User-Agent':generate_user_agent()})
    page = urlopen(req)
    soup = BeautifulSoup(page, 'html.parser')

    task = soup.find('tbody').get_text()

    task_list.append(task)

  2%|▏         | 3/122 [00:01<00:45,  2.62it/s]


AttributeError: 'NoneType' object has no attribute 'get_text'

In [25]:
task_list

['\n\n자율주행 Quality Assurance(QA)\n\n[담당업무]○ 무인 배달 로봇의 자율주행 알고리즘 및 시스템의 현장 검증 및 리포팅○ 다양한 환경에서의 반복 테스트를 통한 엣지 케이스 검출 및 리포팅○ 현장 테스트를 통한 자율주행 SW 시스템 기능의 UI/UX 피드백○ 신규 서비스 구역에 대한 자율주행 매핑 및 주행 검증\n\n\n[자격요건]○ 만 1년 이상 5년 미만의 QA 업무 경력을 보유하신 분○ 테스트 케이스 작성 및 개선 경험○ 리눅스 시스템에 친숙하신 분○ 팀원과의 원활한 소통 및 협업 능력 보유[우대사항]○ 로보틱스, 컴퓨터공학 전공 혹은 그에 준하는 소프트웨어공학 기반 지식 보유○ Linux, ROS, 실내/외 자율주행 알고리즘에 대한 기본 지식 보유○ 테스트 자동화 구축 경험 또는 Python 등 스크립트 언어 작성 경험 있으신 분○ 소통을 위한 협업 tool에 능통하신 분 (Jira, Confluence)\n\n0 명\n\n',
 '\n\n자율주행 Quality Assurance(QA)\n\n[담당업무]○ 무인 배달 로봇의 자율주행 알고리즘 및 시스템의 현장 검증 및 리포팅○ 다양한 환경에서의 반복 테스트를 통한 엣지 케이스 검출 및 리포팅○ 현장 테스트를 통한 자율주행 SW 시스템 기능의 UI/UX 피드백○ 신규 서비스 구역에 대한 자율주행 매핑 및 주행 검증\n\n\n[자격요건]○ 만 1년 이상 5년 미만의 QA 업무 경력을 보유하신 분○ 테스트 케이스 작성 및 개선 경험○ 리눅스 시스템에 친숙하신 분○ 팀원과의 원활한 소통 및 협업 능력 보유[우대사항]○ 로보틱스, 컴퓨터공학 전공 혹은 그에 준하는 소프트웨어공학 기반 지식 보유○ Linux, ROS, 실내/외 자율주행 알고리즘에 대한 기본 지식 보유○ 테스트 자동화 구축 경험 또는 Python 등 스크립트 언어 작성 경험 있으신 분○ 소통을 위한 협업 tool에 능통하신 분 (Jira, Confluence)\n\n0 명\n\n',
 '\n\n자율주행 Quality Ass

In [30]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm 
from user_agent import generate_user_agent

task_list = []

for idx, row in tqdm(autocar_df.iterrows(), total=len(autocar_df.index)):
    req = Request(url=row['링크2'], headers={'User-Agent':generate_user_agent()})
    page = urlopen(req)
    soup = BeautifulSoup(page, 'html.parser')

    tbody_tag = soup.find('tbody')

    # Check if 'tbody' is found
    if tbody_tag:
        task = tbody_tag.get_text()
        task_list.append(task)
    else:
        # Handle the case where 'tbody' is not found
        task_list.append(None)  # or task_list.append('') or any default value you want


100%|██████████| 122/122 [00:20<00:00,  5.85it/s]


### task 데이터 삽입 

In [32]:
autocar_df['task'] = task_list

In [34]:
autocar_df.to_csv("../../data/eda_project_data/autocar3.csv", encoding='utf-8')

### 데이터 가공

In [58]:
autocar_df['담당업무'] = autocar_df['task'].str.split('[').str[1]
autocar_df['자격요견'] = autocar_df['task'].str.split('[').str[2]
autocar_df['우대사항'] = autocar_df['task'].str.split('[').str[3]

In [65]:
autocar_df.to_csv("../../data/eda_project_data/autocar4.csv", encoding='utf-8')

In [66]:

autocar_df.drop(columns=['링크','링크2','task'],inplace=True)

In [68]:
autocar_df.to_csv("../../data/eda_project_data/autocar5.csv", encoding='utf-8')

### 해당 분야의 키워드 설정

In [69]:
autocar_df.head()
keyword =['로보틱스','C','Python','QA','자율주행','Lidar','인공지능','ROS','ROS2','C++','Camera','Tracking','Deep Learning','Object Detection','Object Segmentation'
          ,'석사','박사','리눅스','Linux','컴퓨터공학','소프트웨어공학','ADAS','센서','이미지센서','전기','전자','로봇공학','기계공학','임베디드','컴퓨터','자동차공학','학사','Matlab',
          'Simulink','dSPACE','Vector','Tool','아키텍처','SW','신호처리','영상처리','딥러닝','센서퓨전','제어','마이크로프로세서','운영체제','전자회로','네트워크','동역학','프로젝트','영어',
          '대회','센서','Radar','Platform', '회화', 'Embedded', '문서', '경로 계획', '커뮤니케이션', 'adas', 'AI','ML','DL', 'Geometry','데이터','수학','확률','통계','선형대수','클러스터링'
          , '트리', '알고리즘', 'IT', 'DSP', 'RF','하드웨어','회로','석/박사', '전파','기능안전','펌웨어','Firmware','Safety','AUTOSAR','S/W','레이더','라이더','USPR','Rader', 'Optimization',
          'camera','Localization','자료구조','운영체제','SLAM','IMU','프로그래밍','인프라','플랫폼', '시스템', '서비스', 'vehicle', 'SWITCH','ECU','Battery','Controller','ISO','CE','UL',
          '3D','Robotics','기구 개발','Visual Odometry','Path Planning','Computer vision','Image processing','Architect','모빌리티', 'Modern', 'Semantic segmentation', 
          'Object detection','Navigation', '구동부','제어부','센서부', '영업', 'Control','Autonomous','Verification', 'Issue management', 'AV system', 'Mapping', '3D Reconstruction']

,Unnamed: 0,회사명,모집명,분야,경력,학력,연봉,주소,키워드,담당업무,자격요견,우대사항
0,0,(주)에이알247,자율주행 로봇 Perception 개발자 채용,"빅데이터·AI(인공지능), 응용프로그래머, 소프트웨어·하드웨어",경력(연차무관),대졸이상,"6,000~7,000","(13636) 경기도 성남시 분당구 성남대로 69 (구미동,로드랜드EZ타워) 1층",자율주행,담당업무]자율주행 배달 로봇 Perception 개발\n\n\n,자격요건]경력사항: 경력(연차무관)학력사항: 대학교(4년)졸업 이상직무기술: 자율주...,NaN
1,1,쓰리핸즈,[ 자율주행 Quality Assurance(QA) ] 자율주행 로봇 배달 플랫폼,"소프트웨어·하드웨어, 게임(Game), 응용프로그래머",경력1~5년,대졸,NaN,"(06746) 서울특별시 서초구 바우뫼로 222 (양재동,영진빌딩)",자율주행,담당업무]○ 무인 배달 로봇의 자율주행 알고리즘 및 시스템의 현장 검증 및 리포팅○...,자격요건]○ 만 1년 이상 5년 미만의 QA 업무 경력을 보유하신 분○ 테스트 케이...,"우대사항]○ 로보틱스, 컴퓨터공학 전공 혹은 그에 준하는 소프트웨어공학 기반 지식 ..."
2,2,매버릭컨설팅(주),"PROJECT MANAGER (자동차 자율주행, ADAS)",경영·기획·전략,경력8~22년,대졸,NaN,"(06628) 서울특별시 서초구 강남대로 309 2016호(서초동,코리아비지니스센타)",자율주행,담당업무]자동차 자율주행 분야 PROJECT MANAGER 업무\n\n\n,자격요건]경력사항: 자동차 자율주행\r\n분야 PROJECT 경력(8년 이상 22년...,NaN
3,3,현대자동차(주),자율주행 및 자율주차 시험,"빅데이터·AI(인공지능), 품질관리, 기계·금속·재료",경력(연차무관),대졸이상,NaN,(06797) 서울특별시 서초구 헌릉로 12 (양재동),자율주행,None,None,None
4,4,현대자동차(주),자율주행 및 자율주차 설계,"빅데이터·AI(인공지능), ERP·시스템분석·설계",경력(연차무관),대졸이상,NaN,(06797) 서울특별시 서초구 헌릉로 12 (양재동),자율주행,None,None,None


In [85]:
# 주어진 키워드 리스트
keyword = ['로보틱스', 'C', 'Python', 'QA', '자율주행', 'Lidar', '인공지능', 'ROS', 'ROS2', 'C++', 'Camera', 'Tracking', 'Deep Learning', 'Object Detection', 'Object Segmentation'
          ,'석사','박사','리눅스','Linux','컴퓨터공학','소프트웨어공학','ADAS','센서','이미지센서','전기','전자','로봇공학','기계공학','임베디드','컴퓨터','자동차공학','학사','Matlab',
          'Simulink','dSPACE','Vector','Tool','아키텍처','SW','신호처리','영상처리','딥러닝','센서퓨전','제어','마이크로프로세서','운영체제','전자회로','네트워크','동역학','프로젝트','영어',
          '대회','센서','Radar','Platform', '회화', 'Embedded', '문서', '경로 계획', '커뮤니케이션', 'adas', 'AI','ML','DL', 'Geometry','데이터','수학','확률','통계','선형대수','클러스터링'
          , '트리', '알고리즘', 'IT', 'DSP', 'RF','하드웨어','회로','석/박사', '전파','기능안전','펌웨어','Firmware','Safety','AUTOSAR','S/W','레이더','라이더','USPR','Rader', 'Optimization',
          'camera','Localization','자료구조','운영체제','SLAM','IMU','프로그래밍','인프라','플랫폼', '시스템', '서비스', 'vehicle', 'SWITCH','ECU','Battery','Controller','ISO','CE','UL',
          '3D','Robotics','기구 개발','Visual Odometry','Path Planning','Computer vision','Image processing','Architect','모빌리티', 'Modern', 'Semantic segmentation', 'Object detection'
          ,'Navigation', '구동부','제어부','센서부', '영업', 'Control','Autonomous','Verification', 'Issue management', 'AV system', 'Mapping', '3D Reconstruction']

# '담당업무'에 포함된 키워드만 남기기
autocar_df['담당업무'] = autocar_df['담당업무'].apply(lambda x: ' '.join([word for word in str(x).split() if any(keyword_word.lower() in word.lower() for keyword_word in keyword)]))


In [87]:
autocar_df

,Unnamed: 0,회사명,모집명,분야,경력,학력,연봉,주소,키워드,담당업무,...,제어부,센서부,영업,Control,Autonomous,Verification,Issue management,AV system,Mapping,3D Reconstruction
0,0,(주)에이알247,자율주행 로봇 Perception 개발자 채용,"빅데이터·AI(인공지능), 응용프로그래머, 소프트웨어·하드웨어",경력(연차무관),대졸이상,"6,000~7,000","(13636) 경기도 성남시 분당구 성남대로 69 (구미동,로드랜드EZ타워) 1층",자율주행,담당업무]자율주행 Perception,...,0,0,0,0,0,0,0,0,0,0
1,1,쓰리핸즈,[ 자율주행 Quality Assurance(QA) ] 자율주행 로봇 배달 플랫폼,"소프트웨어·하드웨어, 게임(Game), 응용프로그래머",경력1~5년,대졸,NaN,"(06746) 서울특별시 서초구 바우뫼로 222 (양재동,영진빌딩)",자율주행,자율주행 알고리즘 시스템의 자율주행 SW 시스템 서비스 자율주행,...,0,0,0,0,0,0,0,0,0,0
2,2,매버릭컨설팅(주),"PROJECT MANAGER (자동차 자율주행, ADAS)",경영·기획·전략,경력8~22년,대졸,NaN,"(06628) 서울특별시 서초구 강남대로 309 2016호(서초동,코리아비지니스센타)",자율주행,자율주행 PROJECT,...,0,0,0,0,0,0,0,0,0,0
3,3,현대자동차(주),자율주행 및 자율주차 시험,"빅데이터·AI(인공지능), 품질관리, 기계·금속·재료",경력(연차무관),대졸이상,NaN,(06797) 서울특별시 서초구 헌릉로 12 (양재동),자율주행,,...,0,0,0,0,0,0,0,0,0,0
4,4,현대자동차(주),자율주행 및 자율주차 설계,"빅데이터·AI(인공지능), ERP·시스템분석·설계",경력(연차무관),대졸이상,NaN,(06797) 서울특별시 서초구 헌릉로 12 (양재동),자율주행,,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,117,한국폴리텍대학 성남캠퍼스,[전액무료] 하이테크 과정 교육생 2차 모집,"연구원·연구개발, 기술영업, 바이오·식품공학, 빅데이터·AI(인공지능), 응용프로그...",신입/경력(연차무관),학력무관,NaN,(13122) 경기 성남시 수정구 수정로 398 (산성동),자율주행,,...,0,0,0,0,0,0,0,0,0,0
118,118,(주) 실리콘스튜디오코리아,"2024년 상반기 실리콘스튜디오코리아 부문별 채용[가상현실 엔지니어,AI 엔지니어]","응용프로그래머, 네트워크·서버·보안, 빅데이터·AI(인공지능), ERP·시스템분석·...",신입/경력(연차무관),고졸이상,NaN,(05854) 서울특별시 송파구 송파대로 201 송파 테라타워2 A동 1301호,자율주행,시스템 분IT 네트워크 AI AI AI에 (Python) 프로그래밍 영어,...,0,0,0,0,0,0,0,0,0,0
119,119,한국교통안전공단,자동차안전연구원 자율주행자동차사고조사위원회 사무국 청년인턴 채용 공고,"경영·기획·전략, 일반사무·사무지원, 세무·회계, 법률·법무·회계",신입,학력무관,NaN,NaN,자율주행,,...,0,0,0,0,0,0,0,0,0,0
120,120,현대엠시스템즈(주),"자율주행, 센서퓨전, 임베디드, AI분야 개발자 채용","전자·반도체, 시스템프로그래머",경력2년↑,초대졸,"3,000~8,000",NaN,자율주행,,...,0,0,0,0,0,0,0,0,0,0


In [91]:
autocar2_df = pd.read_csv('../../data/eda_project_data/autocar4.csv')

In [92]:
autocar2_df.drop(columns=['Unnamed: 0.1','Unnamed: 0'], inplace=True)

In [95]:
autocar2_df['task']=autocar2_df['task'].str.split('담당업무').str[1]

In [99]:
autocar2_df.drop(columns=['담당업무','자격요견','우대사항'], inplace=True)

In [101]:
autocar2_df.to_csv('../../data/eda_project_data/autocar6.csv', encoding='utf-8')

In [103]:
autocar2_df['task']=autocar2_df['task'].str.split('자격요건').str[1]

In [105]:
autocar2_df.to_csv('../../data/eda_project_data/autocar6.csv', encoding='utf-8')

In [108]:
keyword = ['로보틱스', 'C', 'Python', 'QA', '자율주행', 'Lidar', '인공지능', 'ROS', 'ROS2', 'C++', 'Camera', 'Tracking', 'Deep Learning', 'Object Detection', 'Object Segmentation'
          ,'석사','박사','리눅스','Linux','컴퓨터공학','소프트웨어공학','ADAS','센서','이미지센서','전기','전자','로봇공학','기계공학','임베디드','컴퓨터','자동차공학','학사','Matlab',
          'Simulink','dSPACE','Vector','Tool','아키텍처','SW','신호처리','영상처리','딥러닝','센서퓨전','제어','마이크로프로세서','운영체제','전자회로','네트워크','동역학','프로젝트','영어',
          '대회','센서','Radar','Platform', '회화', 'Embedded', '문서', '경로 계획', '커뮤니케이션', 'adas', 'AI','ML','DL', 'Geometry','데이터','수학','확률','통계','선형대수','클러스터링'
          , '트리', '알고리즘', 'IT', 'DSP', 'RF','하드웨어','회로','석/박사', '전파','기능안전','펌웨어','Firmware','Safety','AUTOSAR','S/W','레이더','라이더','USPR','Rader', 'Optimization',
          'camera','Localization','자료구조','운영체제','SLAM','IMU','프로그래밍','인프라','플랫폼', '시스템', '서비스', 'vehicle', 'SWITCH','ECU','Battery','Controller','ISO','CE','UL',
          '3D','Robotics','기구 개발','Visual Odometry','Path Planning','Computer vision','Image processing','Architect','모빌리티', 'Modern', 'Semantic segmentation', 'Object detection'
          ,'Navigation', '구동부','제어부','센서부', '영업', 'Control','Autonomous','Verification', 'Issue management', 'AV system', 'Mapping', '소통','3D Reconstruction']

autocar2_df.iloc[1,10]

']○ 만 1년 이상 5년 미만의 QA 업무 경력을 보유하신 분○ 테스트 케이스 작성 및 개선 경험○ 리눅스 시스템에 친숙하신 분○ 팀원과의 원활한 소통 및 협업 능력 보유[우대사항]○ 로보틱스, 컴퓨터공학 전공 혹은 그에 준하는 소프트웨어공학 기반 지식 보유○ Linux, ROS, 실내/외 자율주행 알고리즘에 대한 기본 지식 보유○ 테스트 자동화 구축 경험 또는 Python 등 스크립트 언어 작성 경험 있으신 분○ 소통을 위한 협업 tool에 능통하신 분 (Jira, Confluence)\n\n0 명\n\n'

In [115]:
autocar2_df['task'] = autocar2_df['task'].fillna('')

### keyword 추출 코드

In [169]:
# 키워트 추출 코드
import pandas as pd
import re

# 키워드 리스트
keyword_list = ['로보틱스', 'C', 'Python', 'QA', '자율주행', 'Lidar', '인공지능', 'ROS', 'ROS2', 'C++', 'Camera', 'Tracking', 'Deep Learning', 'Object Detection', 'Object Segmentation'
          ,'석사','박사','리눅스','Linux','컴퓨터공학','소프트웨어공학','ADAS','센서','이미지센서','전기','전자','로봇공학','기계공학','임베디드','컴퓨터','자동차공학','학사','Matlab',
          'Simulink','dSPACE','Vector','Tool','아키텍처','SW','신호처리','영상처리','딥러닝','센서퓨전','제어','마이크로프로세서','운영체제','전자회로','네트워크','동역학','프로젝트','영어',
          '대회','센서','Radar','Platform', '회화', 'Embedded', '문서', '경로 계획', '커뮤니케이션', 'adas', 'AI','ML','DL', 'Geometry','데이터','수학','확률','통계','선형대수','클러스터링'
          , '트리', '알고리즘', 'IT', 'DSP', 'RF','하드웨어','회로','석/박사', '전파','기능안전','펌웨어','Firmware','Safety','AUTOSAR','S/W','레이더','라이더','USPR','Rader', 'Optimization',
          'camera','Localization','자료구조','운영체제','SLAM','IMU','프로그래밍','인프라','플랫폼', '시스템', '서비스', 'vehicle', 'SWITCH','ECU','Battery','Controller','ISO','CE','UL',
          '3D','Robotics','기구 개발','Visual Odometry','Path Planning','Computer vision','Image processing','Architect','모빌리티', 'Modern', 'Semantic segmentation', 'Object detection'
          ,'Navigation', '구동부','제어부','센서부', '영업', 'Control','Autonomous','Verification', 'Issue management', 'AV system', 'Mapping', '소통','3D Reconstruction']

# 특정 열에서 키워드 추출 함수 정의
def extract_keywords_from_column(text, keyword_list):
    extracted_keywords = []
    
    for keyword in keyword_list:
        pattern = r'\b{}\b'.format(re.escape(keyword))
        matches = re.findall(pattern, text, flags=re.IGNORECASE)
        extracted_keywords.extend(matches)
    
    return extracted_keywords

# 특정 열에 키워드 추출 적용
autocar2_df['키워드_추출'] = autocar2_df['task'].apply(lambda x: extract_keywords_from_column(x, keyword_list))



In [114]:
non_string_values = autocar2_df['task'][autocar2_df['task'].apply(lambda x: not isinstance(x, str))]
print(non_string_values)


3      NaN
4      NaN
8      NaN
13     NaN
18     NaN
      ... 
117    NaN
118    NaN
119    NaN
120    NaN
121    NaN
Name: task, Length: 83, dtype: object


In [118]:
autocar2_df.to_csv('../../data/eda_project_data/autocar7.csv')

### test

In [123]:
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import urllib

url = "https://job.incruit.com/jobdb_info/jobpost.asp?job=2401120003740&src=gsw*etc"        # 자신의 찾으려는 웹사이트 입력
res = urlopen(url)
soup = BeautifulSoup(res, "html.parser")



KeyError: 'src'

In [132]:
a = soup.find('div',class_='joview_load_box')
print(a)

None


In [161]:
from tqdm import tqdm 
from user_agent import generate_user_agent

link3_list =[]

for idx, row in tqdm(autocar2_df.iterrows(), total=len(autocar2_df.index)):
    req = Request(url=row['링크'], headers={'User-Agent':generate_user_agent()})
    page = urlopen(req)
    soup = BeautifulSoup(page, 'html.parser')

    a = soup.find('div',class_='joview_load_box')
    a1 = a.find('iframe',id='ifrmJobCont')['src']

    link3_list.append(a1)
    

100%|██████████| 122/122 [01:36<00:00,  1.27it/s]


In [162]:
link3_list 

['https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2205190006235',
 'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2312170000111',
 'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2212230005310',
 'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2401100000202',
 'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2401100000190',
 'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2108170000567',
 'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2208300004159',
 'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2312170000114',
 'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2401100000210',
 'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2302020003951',
 'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2312170000086',
 'https://job.incruit.com/s_common/jobpost/jobpostcont.asp?job=2312170000100',
 'https://job.incruit.com/s_common/jobpost/jobpostco

In [163]:
autocar2_df['링크3'] = link3_list

In [164]:
autocar2_df

,회사명,모집명,링크,분야,경력,학력,연봉,주소,링크2,키워드,task,키워드_추출,링크3
0,(주)에이알247,자율주행 로봇 Perception 개발자 채용,https://job.incruit.com/jobdb_info/jobpost.asp...,"빅데이터·AI(인공지능), 응용프로그래머, 소프트웨어·하드웨어",경력(연차무관),대졸이상,"6,000~7,000","(13636) 경기도 성남시 분당구 성남대로 69 (구미동,로드랜드EZ타워) 1층",https://job.incruit.com/s_common/jobpost/jobpo...,자율주행,]경력사항: 경력(연차무관)학력사항: 대학교(4년)졸업 이상직무기술: 자율주행 SW...,"[자율주행, 자율주행, 자율주행, 인공지능, SW, 시스템]",https://job.incruit.com/s_common/jobpost/jobpo...
1,쓰리핸즈,[ 자율주행 Quality Assurance(QA) ] 자율주행 로봇 배달 플랫폼,https://job.incruit.com/jobdb_info/jobpost.asp...,"소프트웨어·하드웨어, 게임(Game), 응용프로그래머",경력1~5년,대졸,NaN,"(06746) 서울특별시 서초구 바우뫼로 222 (양재동,영진빌딩)",https://job.incruit.com/s_common/jobpost/jobpo...,자율주행,]○ 만 1년 이상 5년 미만의 QA 업무 경력을 보유하신 분○ 테스트 케이스 작성...,"[로보틱스, Python, QA, 자율주행, ROS, 리눅스, Linux, 컴퓨터공...",https://job.incruit.com/s_common/jobpost/jobpo...
2,매버릭컨설팅(주),"PROJECT MANAGER (자동차 자율주행, ADAS)",https://job.incruit.com/jobdb_info/jobpost.asp...,경영·기획·전략,경력8~22년,대졸,NaN,"(06628) 서울특별시 서초구 강남대로 309 2016호(서초동,코리아비지니스센타)",https://job.incruit.com/s_common/jobpost/jobpo...,자율주행,]경력사항: 자동차 자율주행\r\n분야 PROJECT 경력(8년 이상 22년 이하)...,"[자율주행, 자율주행]",https://job.incruit.com/s_common/jobpost/jobpo...
3,현대자동차(주),자율주행 및 자율주차 시험,https://job.incruit.com/jobdb_info/jobpost.asp...,"빅데이터·AI(인공지능), 품질관리, 기계·금속·재료",경력(연차무관),대졸이상,NaN,(06797) 서울특별시 서초구 헌릉로 12 (양재동),https://job.incruit.com/s_common/jobpost/jobpo...,자율주행,,[],https://job.incruit.com/s_common/jobpost/jobpo...
4,현대자동차(주),자율주행 및 자율주차 설계,https://job.incruit.com/jobdb_info/jobpost.asp...,"빅데이터·AI(인공지능), ERP·시스템분석·설계",경력(연차무관),대졸이상,NaN,(06797) 서울특별시 서초구 헌릉로 12 (양재동),https://job.incruit.com/s_common/jobpost/jobpo...,자율주행,,[],https://job.incruit.com/s_common/jobpost/jobpo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,한국폴리텍대학 성남캠퍼스,[전액무료] 하이테크 과정 교육생 2차 모집,https://job.incruit.com/jobdb_info/jobpost.asp...,"연구원·연구개발, 기술영업, 바이오·식품공학, 빅데이터·AI(인공지능), 응용프로그...",신입/경력(연차무관),학력무관,NaN,(13122) 경기 성남시 수정구 수정로 398 (산성동),https://job.incruit.com/s_common/jobpost/jobpo...,자율주행,,[],https://job.incruit.com/s_common/jobpost/jobpo...
118,(주) 실리콘스튜디오코리아,"2024년 상반기 실리콘스튜디오코리아 부문별 채용[가상현실 엔지니어,AI 엔지니어]",https://job.incruit.com/jobdb_info/jobpost.asp...,"응용프로그래머, 네트워크·서버·보안, 빅데이터·AI(인공지능), ERP·시스템분석·...",신입/경력(연차무관),고졸이상,NaN,(05854) 서울특별시 송파구 송파대로 201 송파 테라타워2 A동 1301호,https://job.incruit.com/s_common/jobpost/jobpo...,자율주행,,[],https://job.incruit.com/s_common/jobpost/jobpo...
119,한국교통안전공단,자동차안전연구원 자율주행자동차사고조사위원회 사무국 청년인턴 채용 공고,https://job.incruit.com/jobdb_info/jobpost.asp...,"경영·기획·전략, 일반사무·사무지원, 세무·회계, 법률·법무·회계",신입,학력무관,NaN,NaN,https://job.incruit.com/s_common/jobpost/jobpo...,자율주행,,[],https://job.incruit.com/s_common/jobpost/jobpo...
120,현대엠시스템즈(주),"자율주행, 센서퓨전, 임베디드, AI분야 개발자 채용",https://job.incruit.com/jobdb_info/jobpost.asp...,"전자·반도체, 시스템프로그래머",경력2년↑,초대졸,"3,000~8,000",NaN,https://job.incruit.com/s_common/jobpost/jobpo...,자율주행,,[],https://job.incruit.com/s_common/jobpost/jobpo...


In [165]:
autocar2_df.to_csv('../../data/eda_project_data/autocar8.csv')

In [166]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm 
from user_agent import generate_user_agent

list3 = []

for idx, row in tqdm(autocar2_df.iterrows(), total=len(autocar_df.index)):
    req = Request(url=row['링크3'], headers={'User-Agent':generate_user_agent()})
    page = urlopen(req)
    soup = BeautifulSoup(page, 'html.parser')

    tbody = soup.get_text()
    list3.append(tbody)


100%|██████████| 122/122 [00:22<00:00,  5.33it/s]


In [168]:
autocar2_df['task'] = list3

In [171]:
autocar2_df.to_csv('../../data/eda_project_data/autocar9.csv')

In [174]:
autocar2_df.drop(columns=['링크','링크2','링크3','task'],inplace=True)

In [175]:
autocar2_df

,회사명,모집명,분야,경력,학력,연봉,주소,키워드,키워드_추출
0,(주)에이알247,자율주행 로봇 Perception 개발자 채용,"빅데이터·AI(인공지능), 응용프로그래머, 소프트웨어·하드웨어",경력(연차무관),대졸이상,"6,000~7,000","(13636) 경기도 성남시 분당구 성남대로 69 (구미동,로드랜드EZ타워) 1층",자율주행,"[C, C, 자율주행, 자율주행, 자율주행, 자율주행, 자율주행, 자율주행, 자율주..."
1,쓰리핸즈,[ 자율주행 Quality Assurance(QA) ] 자율주행 로봇 배달 플랫폼,"소프트웨어·하드웨어, 게임(Game), 응용프로그래머",경력1~5년,대졸,NaN,"(06746) 서울특별시 서초구 바우뫼로 222 (양재동,영진빌딩)",자율주행,"[로보틱스, Python, QA, QA, QA, QA, 자율주행, 자율주행, 자율주..."
2,매버릭컨설팅(주),"PROJECT MANAGER (자동차 자율주행, ADAS)",경영·기획·전략,경력8~22년,대졸,NaN,"(06628) 서울특별시 서초구 강남대로 309 2016호(서초동,코리아비지니스센타)",자율주행,"[자율주행, 자율주행, 자율주행, 자율주행, 자율주행, 자율주행, ADAS, ADA..."
3,현대자동차(주),자율주행 및 자율주차 시험,"빅데이터·AI(인공지능), 품질관리, 기계·금속·재료",경력(연차무관),대졸이상,NaN,(06797) 서울특별시 서초구 헌릉로 12 (양재동),자율주행,"[C, C, Python, 자율주행, 자율주행, 자율주행, 자율주행, 자율주행, 자..."
4,현대자동차(주),자율주행 및 자율주차 설계,"빅데이터·AI(인공지능), ERP·시스템분석·설계",경력(연차무관),대졸이상,NaN,(06797) 서울특별시 서초구 헌릉로 12 (양재동),자율주행,"[C, C, Python, 자율주행, 자율주행, 자율주행, 자율주행, 자율주행, 자..."
...,...,...,...,...,...,...,...,...,...
117,한국폴리텍대학 성남캠퍼스,[전액무료] 하이테크 과정 교육생 2차 모집,"연구원·연구개발, 기술영업, 바이오·식품공학, 빅데이터·AI(인공지능), 응용프로그...",신입/경력(연차무관),학력무관,NaN,(13122) 경기 성남시 수정구 수정로 398 (산성동),자율주행,"[C, AI]"
118,(주) 실리콘스튜디오코리아,"2024년 상반기 실리콘스튜디오코리아 부문별 채용[가상현실 엔지니어,AI 엔지니어]","응용프로그래머, 네트워크·서버·보안, 빅데이터·AI(인공지능), ERP·시스템분석·...",신입/경력(연차무관),고졸이상,NaN,(05854) 서울특별시 송파구 송파대로 201 송파 테라타워2 A동 1301호,자율주행,"[Python, 네트워크, 영어, 커뮤니케이션, AI, AI, AI, 프로그래밍, ..."
119,한국교통안전공단,자동차안전연구원 자율주행자동차사고조사위원회 사무국 청년인턴 채용 공고,"경영·기획·전략, 일반사무·사무지원, 세무·회계, 법률·법무·회계",신입,학력무관,NaN,NaN,자율주행,[]
120,현대엠시스템즈(주),"자율주행, 센서퓨전, 임베디드, AI분야 개발자 채용","전자·반도체, 시스템프로그래머",경력2년↑,초대졸,"3,000~8,000",NaN,자율주행,"[자율주행, Linux, 임베디드, 센서퓨전, Embedded, firmware, ..."


In [176]:
autocar2_df['키워드_추출2'] = autocar2_df['키워드_추출'].apply(lambda x: list(set(x)))


In [178]:
import re

# 주소에서 (숫자) 제거하는 함수
def remove_numbers_in_parentheses(address):
    return re.sub(r'\(\d+\)', '', address)

# '주소' 열에 함수 적용
autocar2_df['주소'] = autocar2_df['주소'].apply(remove_numbers_in_parentheses)

# 결과 출력
print(autocar2_df['주소'])


TypeError: expected string or bytes-like object

In [179]:
import re

# 주소에서 (숫자) 제거하는 함수
def remove_numbers_in_parentheses(address):
    if pd.isna(address):
        return address
    return re.sub(r'\(\d+\)', '', address)

# '주소' 열에 함수 적용
autocar2_df['주소'] = autocar2_df['주소'].apply(remove_numbers_in_parentheses)

# 결과 출력
print(autocar2_df['주소'])


0         경기도 성남시 분당구 성남대로 69 (구미동,로드랜드EZ타워) 1층
1                 서울특별시 서초구 바우뫼로 222 (양재동,영진빌딩)
2       서울특별시 서초구 강남대로 309 2016호(서초동,코리아비지니스센타)
3                        서울특별시 서초구 헌릉로 12 (양재동)
4                        서울특별시 서초구 헌릉로 12 (양재동)
                         ...                   
117                    경기 성남시 수정구 수정로 398 (산성동)
118        서울특별시 송파구 송파대로 201 송파 테라타워2 A동 1301호
119                                         NaN
120                                         NaN
121                                         NaN
Name: 주소, Length: 122, dtype: object


In [181]:
autocar2_df.to_csv('../../data/eda_project_data/autocar10.csv')

In [183]:
autocar2_df.drop(columns=['키워드_추출'],inplace=True)

In [185]:
autocar2_df.rename(columns={'회사명':'기업이름',"분야":'담당업무','경력':'고용형태','주소':'기업주소','키워드_추출2':'필요역량'},inplace=True)

In [187]:
autocar2_df.to_csv('../../data/eda_project_data/AutonomousVehicles_tmp.csv', sep=',', encoding='utf-8')

In [189]:
autocar2_df.drop(columns=['모집명','학력'],inplace=True)


In [191]:
new_order = ['기업이름', '고용형태', '연봉', '기업주소', '키워드', '담당업무', '필요역량']
autocar2_df = autocar2_df[new_order]

In [193]:
autocar2_df.to_csv('../../data/eda_project_data/AutonomousVehicles_syt.csv', sep=',', encoding='utf-8')